# Nearest Neighbors

When exploring a large set of documents -- such as Wikipedia, news articles, StackOverflow, etc. -- it can be useful to get a list of related material. To find relevant documents we typically
* Decide on a notion of similarity
* Find the documents that are most similar 

In this notebook we will
* Gain intuition for different notions of similarity and practice finding similar documents. 
* Explore the tradeoffs with representing documents using raw word counts and TF-IDF
* Explore the behavior of different distance metrics by looking at the book recommendations from Joe Dobrow's Dataset ["57,000 Books with Metadata and 'Blurbs'"](https://www.kaggle.com/jdobrow/57000-books-with-metadata-and-blurbs) and their complementarity to answers to a "Big 5 personality traits survey".

**Note to Amazon EC2 users**: To conserve memory, make sure to stop all the other notebooks before running this notebook.

## Download & unzip necessary data

In [ ]:
!wget -O "people_wiki.sframe.zip" "https://d3c33hcgiwev3.cloudfront.net/i5zBDt4bEemhxxJUtZcQoA_7fb43aab245846f09a8c48977883f0b6_people_wiki.sframe.zip?Expires=1617753600&Signature=KKuh1yZT5fslj~6ALSez4KJIhayspaGyJ5~vMXxYsjF28RasYbUw6Hq4aYqObnYQ8J4Hzl1oJ5TY9FO0NIZYMWbaZCkNpjL5DWT01V~ENHhISWSTthQaZGnP7xYANqsnS0X3a4pVOCKHSqo2hzH9cQL0EpRxwA~HUvUbMZDc5ws_&Key-Pair-Id=APKAJLTNE6QMUY6HBC5A"

--2021-04-05 21:23:41--  https://d3c33hcgiwev3.cloudfront.net/i5zBDt4bEemhxxJUtZcQoA_7fb43aab245846f09a8c48977883f0b6_people_wiki.sframe.zip?Expires=1617753600&Signature=KKuh1yZT5fslj~6ALSez4KJIhayspaGyJ5~vMXxYsjF28RasYbUw6Hq4aYqObnYQ8J4Hzl1oJ5TY9FO0NIZYMWbaZCkNpjL5DWT01V~ENHhISWSTthQaZGnP7xYANqsnS0X3a4pVOCKHSqo2hzH9cQL0EpRxwA~HUvUbMZDc5ws_&Key-Pair-Id=APKAJLTNE6QMUY6HBC5A
Resolving d3c33hcgiwev3.cloudfront.net (d3c33hcgiwev3.cloudfront.net)... 13.32.80.109, 13.32.80.173, 13.32.80.218, ...
Connecting to d3c33hcgiwev3.cloudfront.net (d3c33hcgiwev3.cloudfront.net)|13.32.80.109|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 58971371 (56M) [application/zip]
Saving to: ‘people_wiki.sframe.zip’

people_wiki.sframe. 100%[===================>]  56.24M  34.6MB/s    in 1.6s    

2021-04-05 21:23:43 (34.6 MB/s) - ‘people_wiki.sframe.zip’ saved [58971371/58971371]



In [1]:
!wget -O "archive.zip" "https://github.com/TBrockmeyer/big5-book-recommendation/raw/main/archive.zip"

--2021-04-10 08:31:56--  https://github.com/TBrockmeyer/big5-book-recommendation/raw/main/archive.zip
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/TBrockmeyer/big5-book-recommendation/main/archive.zip [following]
--2021-04-10 08:31:57--  https://raw.githubusercontent.com/TBrockmeyer/big5-book-recommendation/main/archive.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19256426 (18M) [application/zip]
Saving to: ‘archive.zip’

archive.zip         100%[===================>]  18.36M  --.-KB/s    in 0.1s    

2021-04-10 08:31:57 (136 MB/s) - ‘archive.zip’ saved [19256426/19256426]



In [2]:
!ls

archive.zip  sample_data


In [ ]:
!unzip people_wiki.sframe.zip

Archive:  people_wiki.sframe.zip
   creating: people_wiki.sframe/
  inflating: people_wiki.sframe/m_cf05efad0f89a530.frame_idx  
   creating: __MACOSX/
   creating: __MACOSX/people_wiki.sframe/
  inflating: __MACOSX/people_wiki.sframe/._m_cf05efad0f89a530.frame_idx  
  inflating: people_wiki.sframe/m_cf05efad0f89a530.0000  
  inflating: __MACOSX/people_wiki.sframe/._m_cf05efad0f89a530.0000  
  inflating: people_wiki.sframe/dir_archive.ini  
  inflating: __MACOSX/people_wiki.sframe/._dir_archive.ini  
  inflating: people_wiki.sframe/m_cf05efad0f89a530.sidx  
  inflating: __MACOSX/people_wiki.sframe/._m_cf05efad0f89a530.sidx  
 extracting: people_wiki.sframe/objects.bin  
  inflating: __MACOSX/people_wiki.sframe/._objects.bin  
  inflating: __MACOSX/._people_wiki.sframe  


In [3]:
!unzip archive.zip

Archive:  archive.zip
  inflating: books_with_blurbs.csv   


## Install required packages

In [4]:
!pip install -U turicreate

     |████████████████████████████████| 92.0MB 40kB/s 
     |████████████████████████████████| 3.5MB 49.8MB/s 
     |████████████████████████████████| 86.4MB 48kB/s 
     |████████████████████████████████| 327kB 42.3MB/s 
     |████████████████████████████████| 3.6MB 40.9MB/s 
     |████████████████████████████████| 3.8MB 40.7MB/s 
     |████████████████████████████████| 450kB 20.2MB/s 
     |████████████████████████████████| 51kB 6.0MB/s 
     |████████████████████████████████| 18.3MB 122kB/s 
  Created wheel for resampy: filename=resampy-0.2.1-cp37-none-any.whl size=320845 sha256=050f42acf12e6e9dbe94f0a87c956bd48cc54e4f59f0cc14290403c9ef040f4e
  Stored in directory: /root/.cache/pip/wheels/ff/4f/ed/2e6c676c23efe5394bb40ade50662e90eb46e29b48324c5f9b
  Created wheel for prettytable: filename=prettytable-0.7.2-cp37-none-any.whl size=13700 sha256=f0153cb4a1de3a4354707702a3a32d9099214ca1b3fb52e340a30fb5b9a08a91
  Stored in directory: /root/.cache/pip/wheels/80/34/1c/3967380d9676d162cb5951

## Import necessary packages

As usual we need to first import the Python packages that we will need.

In [5]:
from __future__ import print_function # to conform python 2.x print to python 3.x
import turicreate
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

## Load Books dataset

We will be using the same dataset of Wikipedia pages that we used in the Machine Learning Foundations course (Course 1). Each element of the dataset consists of a link to the wikipedia article, the name of the person, and the text of the article (in lowercase).  

In [ ]:
wiki = turicreate.SFrame('people_wiki.sframe')

In [121]:
books = turicreate.SFrame('books_with_blurbs.csv')

Unexpected characters after last column. "The World's Stupidest Signs"
Parse failed at token ending at: 
	f his own strength."
1854795554,The World's Stupidest Signs,^Michael O'Mara,2000,Michael O'Mara Books,"A biza
Successfully parsed 6 tokens: 
	0: 140292918
	1: Of Mice an ... ssentials"
	2: John Steinbeck
	3: 2001
	4: Penguin Books Ltd
	5: As drifter ...  strength.

Unexpected characters after last column. ""
Parse failed at token ending at: 
	 revisits the territory covered in his young adult novel, ,,^ which was likewise set in the hardscrabble worl
Successfully parsed 6 tokens: 
	0: 
	1: Much of
	2: -- winner  ... ng novella
	3: 
	4: 
	5:

Unexpected characters after last column. "Kushiel's Avatar (Kushiel's Legacy)"
Parse failed at token ending at: 
	man nature."
0765347539,Kushiel's Avatar (Kushiel's Legacy),^Jacqueline Carey,2004,Tor Fantasy,"The land of T
Successfully parsed 6 tokens: 
	0: 449003787
	1: \O"
	2: SUE GRAFTON
	3: 2001
	4: Ballantine Books
	5: Through fo ... an nature.

Unexpected characters after last column. "The Strong-Willed Child: Birth Through Adolescence"
Parse failed at token ending at: 
	4236661X,The Strong-Willed Child: Birth Through Adolescence,^James Dobson,1985,Tyndale House Publishers,"Dr D
Successfully parsed 6 tokens: 
	0: 1558743669
	1: A Child Called \It"
	2: Dave Pelzer
	3: 1995
	4: Health Communications
	5:  ,This boo ... their son.

Unexpected characters after last column. "and tapes of the complete plays.  "
Parse failed at token ending at: 
	ideotapes (VHS), records, and tapes of the complete plays.  
Successfully parsed 6 tokens: 
	0: Background ... dramaturgy
	1: C. W. Hodg ...  Playhouse
	2: a brief ac ...  his works
	3: and a bibliography
	4: lists of v ... apes (VHS)
	5: records

...

Successfully parsed 1 tokens: 
	0: Format: Hardcover

Successfully parsed 1 tokens: 
	0: Publicatio ...  3/12/1957

327 lines failed to parse correctly

Finished parsing file /content/books_with_blurbs.csv

Parsing completed. Parsed 25671 lines in 0.612788 secs.

In [122]:
books[:3]

ISBN,Title,Author,Year,Publisher,Blurb
0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,"Here, for the first timein paperback, is an ..."
0374157065,Flu: The Story of theGreat Influenza Pandemic ...,Gina Bari Kolata,1999,Farrar Straus Giroux,"The fascinating, truestory of the world's ..."
0399135782,The Kitchen God's Wife,Amy Tan,1991,Putnam Pub Group,Winnie and Helen havekept each others worst ...


Remove stopwords, empty rows and duplicates (keeping the newest edition):

In [123]:
books['Blurb'] = turicreate.text_analytics.drop_words(books['Blurb'], threshold=1, to_lower=True, delimiters=['\r', '\x0b', '\n', '\x0c', '\t', ' ', '!', '#', '$', '%', '&', "'", '"', '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~'], stop_words=turicreate.text_analytics.stop_words())

In [124]:
print(len(books))
books = books.sort(['ISBN']).sort(['Year'], ascending=False).drop_duplicates(subset=['Author','Title'])
print(len(books))

25671
24792


## Extract word count vectors

As we have seen in Course 1, we can extract word count vectors using a Turi Create utility function.  We add this as a column in `wiki` and in `books`.

In [ ]:
wiki['word_count'] = turicreate.text_analytics.count_words(wiki['text'])

In [126]:
books['word_count'] = turicreate.text_analytics.count_words(books['Blurb'])

In [ ]:
wiki

URI,name,text,word_count
<http://dbpedia.org/resource/Digby_Morrell> ...,Digby Morrell,digby morrell born 10october 1979 is a former ...,"{'melbourne': 1.0,'parade': 1.0, ..."
<http://dbpedia.org/resource/Alfred_J._Lewy> ...,Alfred J. Lewy,alfred j lewy aka sandylewy graduated from ...,"{'time': 1.0, 'each':1.0, 'hour': 1.0, ..."
<http://dbpedia.org/resource/Harpdog_Brown> ...,Harpdog Brown,harpdog brown is a singerand harmonica player who ...,"{'society': 1.0,'hamilton': 1.0, 'to': ..."
<http://dbpedia.org/resource/Franz_Rottensteiner> ...,Franz Rottensteiner,franz rottensteiner bornin waidmannsfeld lower ...,"{'kurdlawitzpreis': 1.0,'awarded': 1.0, '2004': ..."
<http://dbpedia.org/resource/G-Enka> ...,G-Enka,henry krvits born 30december 1974 in tallinn ...,"{'curtis': 1.0, '2007':1.0, 'cent': 1.0, ..."
<http://dbpedia.org/resource/Sam_Henderson> ...,Sam Henderson,sam henderson bornoctober 18 1969 is an ...,"{'asses': 1.0, 'sic':1.0, 'toilets': 1.0, ..."
<http://dbpedia.org/resource/Aaron_LaCrate> ...,Aaron LaCrate,aaron lacrate is anamerican music producer ...,"{'streamz': 1.0,'including': 1.0, ..."
<http://dbpedia.org/resource/Trevor_Ferguson> ...,Trevor Ferguson,trevor ferguson aka johnfarrow born 11 november ...,"{'concordia': 1.0,'creative': 1.0, ..."
<http://dbpedia.org/resource/Grant_Nelson> ...,Grant Nelson,grant nelson born 27april 1971 in london ...,"{'heavies': 1.0, 'new':1.0, 'brand': 1.0, ..."
<http://dbpedia.org/resource/Cathy_Caruth> ...,Cathy Caruth,cathy caruth born 1955 isfrank h t rhodes ...,"{'2002': 1.0, 'harvard':1.0, 'twentieth': 1.0, ..."


In [17]:
books

ISBN,Title,Author,Year,Publisher
0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
0374157065,Flu: The Story of theGreat Influenza Pandemic ...,Gina Bari Kolata,1999,Farrar Straus Giroux
0399135782,The Kitchen God's Wife,Amy Tan,1991,Putnam Pub Group
0425176428,What If?: The World'sForemost Military ...,Robert Cowley,2000,Berkley Publishing Group
1881320189,Goodbye to the ButtermilkSky ...,Julia Oliver,1994,River City Pub
0440234743,The Testament,John Grisham,1999,Dell
0452264464,Beloved (PlumeContemporary Fiction) ...,Toni Morrison,1994,Plume
0609804618,Our Dumb Century: TheOnion Presents 100 Years ...,The Onion,1999,Three Rivers Press
1841721522,New Vegetarian: Bold andBeautiful Recipes for ...,Celia Brooks Brown,2001,Ryland Peters &amp; SmallLtd ...
0439095026,Tell Me This Isn'tHappening ...,Robynn Clairday,1999,Scholastic


## Find nearest neighbors

Let's start by
- finding the nearest neighbors of the Barack Obama page using the word count vectors to represent the articles and Euclidean distance to measure distance.
- finding the nearest neighbors for a certain combination of "Big 5" personality traits, using the word count vectors to represent the book blurbs and Euclidean distance to measure distance.

For this, again will we use a Turi Create implementation of nearest neighbor search.

In [ ]:
model_wiki = turicreate.nearest_neighbors.create(wiki, label='name', features=['word_count'],
                                            method='brute_force', distance='euclidean')

Starting brute force nearest neighbors model training.

Validating distance components.

Initializing model data.

Initializing distances.

Done.

In [ ]:
model_books = turicreate.nearest_neighbors.create(books, label='Title', features=['word_count'],
                                            method='brute_force', distance='cosine')

Starting brute force nearest neighbors model training.

Validating distance components.

Initializing model data.

Initializing distances.

Done.

Let's look at the top 10 nearest neighbors by performing the following query:

In [ ]:
model_wiki.query(wiki[wiki['name']=='Barack Obama'], label='name', k=10)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 7.735ms      |

| Done         |         | 100         | 478.357ms    |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
Barack Obama,Barack Obama,0.0,1
Barack Obama,Joe Biden,33.075670817082454,2
Barack Obama,George W. Bush,34.39476704383968,3
Barack Obama,Lawrence Summers,36.15245496505044,4
Barack Obama,Mitt Romney,36.16628264005025,5
Barack Obama,Francisco Barrio,36.3318042491699,6
Barack Obama,Walter Mondale,36.40054944640259,7
Barack Obama,Wynn Normington Hugh-Jones ...,36.49657518178932,8
Barack Obama,Don Bonker,36.6333181680284,9
Barack Obama,Andy Anstett,36.959437225152655,10


In [ ]:
books[books['Title']=='The Testament']

Columns:
	ISBN	str
	Title	str
	Author	str
	Year	int
	Publisher	str
	Blurb	str
	word_count	dict

Rows: Unknown

Data:
+------------+---------------+--------------+------+-----------+
|    ISBN    |     Title     |    Author    | Year | Publisher |
+------------+---------------+--------------+------+-----------+
| 0440234743 | The Testament | John Grisham | 1999 |    Dell   |
+------------+---------------+--------------+------+-----------+
+-------------------------------+-------------------------------+
|             Blurb             |           word_count          |
+-------------------------------+-------------------------------+
| plush virginia office rich... | {'holds': 1.0, 'friends': ... |
+-------------------------------+-------------------------------+
[? rows x 7 columns]
Note: Only the head of the SFrame is printed. This SFrame is lazily evaluated.
You can use sf.materialize() to force materialization.

In [ ]:
model_books.query(books[books['Title']=='The Testament'], label='Title', k=10)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00389545  | 17.162ms     |

| Done         |         | 100         | 86.422ms     |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
The Testament,The Testament,-2.220446049250313e-16,1
The Testament,Writ of Execution,0.8229279382562928,2
The Testament,Looking for RachelWallace ...,0.832284205186034,3
The Testament,GIFT OF STONES,0.8573045517536517,4
The Testament,"Here's to You, RachelRobinson ...",0.8582281565926204,5
The Testament,On a Pale Horse(Incarnations of ...,0.8586832840702012,6
The Testament,Astrology: Woman to Woman,0.860846924537976,7
The Testament,DANIELS BRIDE,0.8613249509436927,8
The Testament,Atlas Shrugged,0.8627187054032712,9
The Testament,Time to Die,0.8650236188002458,10


All of the 10 people are politicians, but about half of them have rather tenuous connections with Obama, other than the fact that they are politicians.

* Francisco Barrio is a Mexican politician, and a former governor of Chihuahua.
* Walter Mondale and Don Bonker are Democrats who made their career in late 1970s.
* Wynn Normington Hugh-Jones is a former British diplomat and Liberal Party official.
* Andy Anstett is a former politician in Manitoba, Canada.

But none of the top 10 books are crime or legal novels.

Nearest neighbors with raw word counts got some things right, showing all politicians in the query result, but missed similarity for the books.

For instance, let's find out why Francisco Barrio was considered a close neighbor of Obama; and why "Teach Yourself Russian" was considered near to "The Testament". To do this, let's look at the most frequently used words in each:

In [ ]:
def top_words(dataset, query_column, query_term):
    """
    Get a table of the most frequent words in the given person's wikipedia page.
    """
    row = dataset[dataset[query_column] == query_term]
    print(row)
    word_count_table = row[['word_count']].stack('word_count', new_column_name=['word','count'])
    return word_count_table.sort('count', ascending=False)

In [ ]:
obama_words = top_words(wiki, 'name', 'Barack Obama')
obama_words

word,count
the,40.0
in,30.0
and,21.0
of,18.0
to,14.0
his,11.0
obama,9.0
act,8.0
a,7.0
he,7.0


In [ ]:
barrio_words = top_words(wiki, 'name', 'Francisco Barrio')
barrio_words

word,count
the,36.0
of,24.0
and,18.0
in,17.0
he,10.0
to,9.0
chihuahua,7.0
a,6.0
governor,6.0
his,5.0


In [ ]:
thetestament_words = top_words(books, 'Title', 'The Testament')
thetestament_words

+------------+---------------+--------------+------+-----------+
|    ISBN    |     Title     |    Author    | Year | Publisher |
+------------+---------------+--------------+------+-----------+
| 0440234743 | The Testament | John Grisham | 1999 |    Dell   |
+------------+---------------+--------------+------+-----------+
+-------------------------------+-------------------------------+
|             Blurb             |           word_count          |
+-------------------------------+-------------------------------+
| plush virginia office rich... | {'holds': 1.0, 'friends': ... |
+-------------------------------+-------------------------------+
[? rows x 7 columns]
Note: Only the head of the SFrame is printed. This SFrame is lazily evaluated.
You can use sf.materialize() to force materialization.


word,count
phelan,3.0
nate,2.0
surprise,2.0
death,2.0
message,2.0
rachel,2.0
lane,2.0
troy,2.0
woman,2.0
man,1.0


In [ ]:
atlasshrugged_words = top_words(books, 'Title', 'Atlas Shrugged')
atlasshrugged_words

+------------+----------------+----------+------+-------------+
|    ISBN    |     Title      |  Author  | Year |  Publisher  |
+------------+----------------+----------+------+-------------+
| 0451191145 | Atlas Shrugged | Ayn Rand | 1996 | Signet Book |
| 0452011876 | Atlas Shrugged | Ayn Rand | 1999 | Plume Books |
+------------+----------------+----------+------+-------------+
+-------------------------------+-------------------------------+
|             Blurb             |           word_count          |
+-------------------------------+-------------------------------+
| story man stop motor world... | {'rebirth': 1.0, 'body': 1... |
| fight battle enemies neede... | {'impossible': 1.0, 'ruthl... |
+-------------------------------+-------------------------------+
[? rows x 7 columns]
Note: Only the head of the SFrame is printed. This SFrame is lazily evaluated.
You can use sf.materialize() to force materialization.


word,count
—,9.0
man,4.0
story,3.0
world,2.0
story,2.0
motor,2.0
woman,2.0
battle,2.0
events,2.0
battle,2.0


Let's extract the list of most frequent words that appear in both Obama's and Barrio's documents. We've so far sorted all words from Obama and Barrio's articles by their word frequencies. We will now use a dataframe operation known as **join**. The **join** operation is very useful when it comes to playing around with data: it lets you combine the content of two tables using a shared column (in this case, the word column). See [the documentation](https://dato.com/products/create/docs/generated/graphlab.SFrame.join.html) for more details.

For instance, running
```
obama_words.join(barrio_words, on='word')
```
will extract the rows from both tables that correspond to the common words.

In [ ]:
combined_words = obama_words.join(barrio_words, on='word')
combined_words

Since both tables contained the column named `count`, SFrame automatically renamed one of them to prevent confusion. Let's rename the columns to tell which one is for which. By inspection, we see that the first column (`count`) is for Obama and the second (`count.1`) for Barrio.

In [ ]:
combined_words = combined_words.rename({'count':'Obama', 'count.1':'Barrio'})
combined_words

**Note**. The **join** operation does not enforce any particular ordering on the shared column. So to obtain, say, the five common words that appear most often in Obama's article, sort the combined table by the Obama column. Don't forget `ascending=False` to display largest counts first.

In [ ]:
combined_words.sort('Obama', ascending=False)

**Quiz Question**. Among the words that appear in both Barack Obama and Francisco Barrio, take the 5 that appear most frequently in Obama. How many of the articles in the Wikipedia dataset contain all of those 5 words?

Hint:
* Refer to the previous paragraph for finding the words that appear in both articles. Sort the common words by their frequencies in Obama's article and take the largest five.
* Each word count vector is a Python dictionary. For each word count vector in SFrame, you'd have to check if the set of the 5 common words is a subset of the keys of the word count vector. Complete the function `has_top_words` to accomplish the task.
  - Convert the list of top 5 words into set using the syntax
```
set(common_words)
```
    where `common_words` is a Python list. See [this link](https://docs.python.org/2/library/stdtypes.html#set) if you're curious about Python sets.
  - Extract the list of keys of the word count dictionary by calling the [`keys()` method](https://docs.python.org/2/library/stdtypes.html#dict.keys).
  - Convert the list of keys into a set as well.
  - Use [`issubset()` method](https://docs.python.org/2/library/stdtypes.html#set) to check if all 5 words are among the keys.
* Now apply the `has_top_words` function on every row of the SFrame.
* Compute the sum of the result column to obtain the number of articles containing all the 5 top words.

In [ ]:
common_words = ... # YOUR CODE HERE

def has_top_words(word_count_vector):
    # extract the keys of word_count_vector and convert it to a set
    unique_words = ...   # YOUR CODE HERE
    # return True if common_words is a subset of unique_words
    # return False otherwise
    return ...  # YOUR CODE HERE

wiki['has_top_words'] = wiki['word_count'].apply(has_top_words)

# use has_top_words column to answer the quiz question
... # YOUR CODE HERE

**Checkpoint**. Check your `has_top_words` function on two random articles:

In [ ]:
print('Output from your function:', has_top_words(wiki[32]['word_count']))
print('Correct output: True')
print('Also check the length of unique_words. It should be 167')
print(len(wiki[32]['word_count']))

In [ ]:
print('Output from your function:', has_top_words(wiki[33]['word_count']))
print('Correct output: False')
print('Also check the length of unique_words. It should be 188')
print(len(wiki[33]['word_count']))

**Quiz Question**. Measure the pairwise distance between the Wikipedia pages of Barack Obama, George W. Bush, and Joe Biden. Which of the three pairs has the smallest distance?

Hints: 
* To compute the Euclidean distance between two dictionaries, use `turicreate.toolkits.distances.euclidean`. Refer to [this link](https://apple.github.io/turicreate/docs/api/generated/turicreate.toolkits.distances.euclidean.html) for usage.
* When using Boolean filter in SFrame/SArray, take the index 0 to access the first match. (Round your answer to three decimal places.)

**Quiz Question**. Collect all words that appear both in Barack Obama and George W. Bush pages.  Out of those words, find the 10 words that show up most often in Obama's page.

**Note.** Even though common words are swamping out important subtle differences, commonalities in rarer political words still matter on the margin. This is why politicians are being listed in the query result instead of musicians, for example. In the next subsection, we will introduce a different metric that will place greater emphasis on those rarer words.

## TF-IDF to the rescue

Much of the perceived commonalities between Obama and Barrio were due to occurrences of extremely frequent words, such as "the", "and", and "his". So nearest neighbors is recommending plausible results sometimes for the wrong reasons. 

To retrieve articles that are more relevant, we should focus more on rare words that don't happen in every article. **TF-IDF** (term frequency–inverse document frequency) is a feature representation that penalizes words that are too common.  Let's use Turi Create's implementation of TF-IDF and repeat the search for the 10 nearest neighbors of Barack Obama:

In [ ]:
wiki['tf_idf'] = turicreate.text_analytics.tf_idf(wiki['word_count'])

In [127]:
books['tf_idf'] = turicreate.text_analytics.tf_idf(books['word_count'])

In [ ]:
model_wiki_tf_idf = turicreate.nearest_neighbors.create(wiki, label='name', features=['tf_idf'],
                                                   method='brute_force', distance='euclidean')

Starting brute force nearest neighbors model training.

Validating distance components.

Initializing model data.

Initializing distances.

Done.

In [128]:
model_books_tf_idf = turicreate.nearest_neighbors.create(books, label='Title', features=['tf_idf'],
                                                   method='brute_force', distance='cosine')

Starting brute force nearest neighbors model training.

Validating distance components.

Initializing model data.

Initializing distances.

Done.

In [ ]:
model_wiki_tf_idf.query(wiki[wiki['name'] == 'Barack Obama'], label='name', k=10)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 14.201ms     |

| Done         |         | 100         | 557.228ms    |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
Barack Obama,Barack Obama,0.0,1
Barack Obama,Phil Schiliro,106.86101369140928,2
Barack Obama,Jeff Sessions,108.87167421571077,3
Barack Obama,Jesse Lee (politician),109.04569790902957,4
Barack Obama,Samantha Power,109.10810616502708,5
Barack Obama,Bob Menendez,109.78186710530214,6
Barack Obama,Eric Stern (politician),109.9577880796839,7
Barack Obama,James A. Guest,110.41388871759888,8
Barack Obama,Roland Grossenbacher,110.47060870018983,9
Barack Obama,Tulsi Gabbard,110.69699799880009,10


## This query shows that our model works quite well right now:
With cosine distance, the found novels are quite near to the original one.

Add a searchterm to the dataset, and calculate its tf_idf scores within the whole of the documents

In [129]:
searchterm_title = "Searchterm 001"

searchterm_blurb = "Introverted, Calm, Aloof, Modest"

searchterm_dict = {
    'ISBN': ["0000000000"],
    'Title': [searchterm_title],
    'Author': ["Admin Admin"],
    'Year': [2021],
    'Publisher': ["None"],
    'Blurb': [searchterm_blurb]
}

searchterm_sf = turicreate.SFrame(searchterm_dict, format='dict')

searchterm_sf['word_count'] = turicreate.text_analytics.count_words(searchterm_sf['Blurb'])
searchterm_sf['tf_idf'] = turicreate.text_analytics.tf_idf(searchterm_sf['word_count'])

books_2 = books.append(searchterm_sf)

books_2['tf_idf'] = turicreate.text_analytics.tf_idf(books_2['word_count'])

Search for books that correspond to the searchterm blurb.

In [138]:
result = model_books_tf_idf.query(books_2[books_2['Title'] == searchterm_title], label='Title', k=10)

result_merged = result.rename({'reference_label': 'Title'}).join(books['ISBN','Title','Author','Year','Publisher','Blurb'], how="left", on="Title")

result_merged = result_merged.sort('rank')
result_merged

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00403356  | 21.787ms     |

| Done         |         | 100         | 82.69ms      |

+--------------+---------+-------------+--------------+

query_label,Title,distance,rank,ISBN,Author,Year
Searchterm 001,A Separate Peace,0.7651588075872895,1,055323224X,John Knowles,1975
Searchterm 001,Lord of Ice,0.8749229724343938,2,0804119732,Gaelen Foley,2002
Searchterm 001,Inner Peace: How to BeCalmly Active and ...,0.8828790667171713,3,0876120109,Yogananda,1999
Searchterm 001,Far from the MaddingCrowd (Wordsworth ...,0.886167724766866,4,1853260673,Thomas Hardy,1997
Searchterm 001,Don't Sweat the SmallStuff and It's All Small ...,0.8976688090397072,5,0786881852,Richard Carlson,1997
Searchterm 001,Don't Sweat the SmallStuff... and It's All ...,0.8976688090397072,6,0786864249,Richard Carlson,1997
Searchterm 001,Cure for a Charlatan,0.9049756761713669,7,0425167348,Caroline Roe,1999
Searchterm 001,Jane Eyre (Miramax Books),0.9088444101454728,8,0786881186,Charlotte Bronte,1996
Searchterm 001,The Horse Whisperer,0.9104827331625444,9,0385315236,Nicholas Evans,1995
Searchterm 001,The Homecoming,0.9110489728037301,10,0843943521,Kimberly Rangel,1998


Let's determine whether this list makes sense.
* With a notable exception of Roland Grossenbacher, the other 8 are all American politicians who are contemporaries of Barack Obama.
* Phil Schiliro, Jesse Lee, Samantha Power, and Eric Stern worked for Obama.

Clearly, the results are more plausible with the use of TF-IDF. Let's take a look at the word vector for Obama and Schilirio's pages. Notice that TF-IDF representation assigns a weight to each word. This weight captures relative importance of that word in the document. Let us sort the words in Obama's article by their TF-IDF weights; we do the same for Schiliro's article as well.

In [131]:
def top_words_tf_idf(dataset, query_column, query_term):
    row = dataset[dataset[query_column] == query_term]
    word_count_table = row[['tf_idf']].stack('tf_idf', new_column_name=['word','weight'])
    return word_count_table.sort('weight', ascending=False)

In [139]:
searchterm_tf_idf = top_words_tf_idf(books_2, 'Title', searchterm_title)
searchterm_tf_idf

word,weight
introverted,9.019704345578143
aloof,7.345727912006471
modest,6.454754988116606
calm,6.2062936288181065


Using the **join** operation we learned earlier, try your hands at computing the common words shared by Obama's and Schiliro's articles. Sort the common words by their TF-IDF weights in Obama's document.

In [142]:
searchterm_tf_idf = top_words_tf_idf(books_2, 'Title', searchterm_title)
searchterm_tf_idf

resultterm_tf_idf = top_words_tf_idf(books_2, 'Title', result_merged['Title'][0])
resultterm_tf_idf

searchterm_tf_idf = searchterm_tf_idf.join(resultterm_tf_idf, how="left", on='word')
author_title = result_merged['Author'][0] + ": " + result_merged['Title'][0]
searchterm_tf_idf = searchterm_tf_idf.rename({'weight':searchterm_blurb, 'weight.1':author_title})

for i in range (1, len(result_merged)):
  resultterm_tf_idf = top_words_tf_idf(books_2, 'Title', result_merged['Title'][i])
  resultterm_tf_idf
  searchterm_tf_idf = searchterm_tf_idf.join(resultterm_tf_idf, how="left", on='word')
  author_title = result_merged['Author'][i] + ": " + result_merged['Title'][i]
  searchterm_tf_idf = searchterm_tf_idf.rename({'weight':author_title})
searchterm_tf_idf

Columns:
	word	str
	Introverted, Calm, Aloof, Modest	float
	John Knowles: A Separate Peace	float
	Gaelen Foley: Lord of Ice	float
	Yogananda: Inner Peace: How to Be Calmly Active and Actively Calm	float
	Thomas Hardy: Far from the Madding Crowd (Wordsworth Classics)	float
	Richard Carlson: Don't Sweat the Small Stuff and It's All Small Stuff : Simple Ways to Keep the Little Things from Taking Over Your Life (Don't Sweat the Small Stuff Series)	float
	Richard Carlson: Don't Sweat the Small Stuff... and It's All Small Stuff: Simple Ways to Keep the Little Things from Taking Over Your Life	float
	Caroline Roe: Cure for a Charlatan	float
	Charlotte Bronte: Jane Eyre (Miramax Books)	float
	Nicholas Evans: The Horse Whisperer	float
	Kimberly Rangel: The Homecoming	float
	Harold Pinter: The Homecoming	float

Rows: 4

Data:
+-------------+-------------------------------+--------------------------------+
|     word    | Introverted, Calm, Aloof, ... | John Knowles: A Separate Peace |
+-------------+-------------------------------+--------------------------------+
|    aloof    |       7.345727912006471       |              None              |
|    modest   |       6.454754988116606       |              None              |
| introverted |       9.019704345578143       |       9.019704345578143        |
|     calm    |       6.2062936288181065      |              None              |
+-------------+-------------------------------+--------------------------------+
+---------------------------+-------------------------------+-------------------------------+
| Gaelen Foley: Lord of Ice | Yogananda: Inner Peace: Ho... | Thomas Hardy: Far from the... |
+---------------------------+-------------------------------+-------------------------------+
|     14.691455824012943    |              None             |              None             |
|            None           |              None             |              None             |
|            None           |              None             |       9.019704345578143       |
|            None           |       6.2062936288181065      |              None             |
+---------------------------+-------------------------------+-------------------------------+
+-------------------------------+-------------------------------+
| Richard Carlson: Don't Swe... | Richard Carlson: Don't Swe... |
+-------------------------------+-------------------------------+
|              None             |              None             |
|              None             |              None             |
|              None             |              None             |
|       12.412587257636213      |       12.412587257636213      |
+-------------------------------+-------------------------------+
+-------------------------------+-------------------------------+
| Caroline Roe: Cure for a C... | Charlotte Bronte: Jane Eyr... |
+-------------------------------+-------------------------------+
|              None             |              None             |
|              None             |              None             |
|              None             |              None             |
|       6.2062936288181065      |       6.2062936288181065      |
+-------------------------------+-------------------------------+
+-------------------------------+-------------------------------+
| Nicholas Evans: The Horse ... | Kimberly Rangel: The Homec... |
+-------------------------------+-------------------------------+
|              None             |       7.345727912006471       |
|              None             |              None             |
|              None             |              None             |
|       6.2062936288181065      |              None             |
+-------------------------------+-------------------------------+
+-------------------------------+
| Harold Pinter: The Homecoming |
+-------------------------------+
|       7.345727912006471       |
|              None             |
|              Non

The first 10 words should say: Obama, law, democratic, Senate, presidential, president, policy, states, office, 2011.

**Quiz Question**. Among the words that appear in both Barack Obama and Phil Schiliro, take the 5 that have largest weights in Obama. How many of the articles in the Wikipedia dataset contain all of those 5 words?

In [ ]:
common_words = ...  # YOUR CODE HERE

def has_top_words(word_count_vector):
    # extract the keys of word_count_vector and convert it to a set
    unique_words = ...   # YOUR CODE HERE
    # return True if common_words is a subset of unique_words
    # return False otherwise
    return ...  # YOUR CODE HERE

wiki['has_top_words'] = wiki['word_count'].apply(has_top_words)

# use has_top_words column to answer the quiz question
...  # YOUR CODE HERE

Notice the huge difference in this calculation using TF-IDF scores instead  of raw word counts. We've eliminated noise arising from extremely common words.

## Choosing metrics

You may wonder why Joe Biden, Obama's running mate in two presidential elections, is missing from the query results of `model_tf_idf`. Let's find out why. First, compute the distance between TF-IDF features of Obama and Biden.

**Quiz Question**. Compute the Euclidean distance between TF-IDF features of Obama and Biden. Recall: When using Boolean filter in SFrame/SArray, take the index 0 to access the first match. (Round your answer to three decimal places.)

The distance is larger than the distances we found for the 10 nearest neighbors, which we repeat here for readability:

In [ ]:
model_tf_idf.query(wiki[wiki['name'] == 'Barack Obama'], label='name', k=10)

But one may wonder, is Biden's article that different from Obama's, more so than, say, Schiliro's? It turns out that, when we compute nearest neighbors using the Euclidean distances, we unwittingly favor short articles over long ones. Let us compute the length of each Wikipedia document, and examine the document lengths for the 100 nearest neighbors to Obama's page.

In [ ]:
def compute_length(row):
    return len(row['text'].split(' '))

wiki['length'] = wiki.apply(compute_length) 

In [ ]:
nearest_neighbors_euclidean = model_tf_idf.query(wiki[wiki['name'] == 'Barack Obama'], label='name', k=100)
nearest_neighbors_euclidean = nearest_neighbors_euclidean.join(wiki[['name', 'length']], on={'reference_label':'name'})

In [ ]:
nearest_neighbors_euclidean.sort('rank')

To see how these document lengths compare to the lengths of other documents in the corpus, let's make a histogram of the document lengths of Obama's 100 nearest neighbors and compare to a histogram of document lengths for all documents.

In [ ]:
plt.figure(figsize=(10.5,4.5))
plt.hist(wiki['length'], 50, color='k', edgecolor='None', histtype='stepfilled', normed=True,
         label='Entire Wikipedia', zorder=3, alpha=0.8)
plt.hist(nearest_neighbors_euclidean['length'], 50, color='r', edgecolor='None', histtype='stepfilled', normed=True,
         label='100 NNs of Obama (Euclidean)', zorder=10, alpha=0.8)
plt.axvline(x=wiki['length'][wiki['name'] == 'Barack Obama'][0], color='k', linestyle='--', linewidth=4,
           label='Length of Barack Obama', zorder=2)
plt.axvline(x=wiki['length'][wiki['name'] == 'Joe Biden'][0], color='g', linestyle='--', linewidth=4,
           label='Length of Joe Biden', zorder=1)
plt.axis([0, 1000, 0, 0.04])

plt.legend(loc='best', prop={'size':15})
plt.title('Distribution of document length')
plt.xlabel('# of words')
plt.ylabel('Percentage')
plt.rcParams.update({'font.size':16})
plt.tight_layout()

Relative to the rest of Wikipedia, nearest neighbors of Obama are overwhemingly short, most of them being shorter than 300 words. The bias towards short articles is not appropriate in this application as there is really no reason to  favor short articles over long articles (they are all Wikipedia articles, after all). Many of the Wikipedia articles are 300 words or more, and both Obama and Biden are over 300 words long.

**Note**: For the interest of computation time, the dataset given here contains _excerpts_ of the articles rather than full text. For instance, the actual Wikipedia article about Obama is around 25000 words. Do not be surprised by the low numbers shown in the histogram.

**Note:** Both word-count features and TF-IDF are proportional to word frequencies. While TF-IDF penalizes very common words, longer articles tend to have longer TF-IDF vectors simply because they have more words in them.

To remove this bias, we turn to **cosine distances**:
$$
d(\mathbf{x},\mathbf{y}) = 1 - \frac{\mathbf{x}^T\mathbf{y}}{\|\mathbf{x}\| \|\mathbf{y}\|}
$$
Cosine distances let us compare word distributions of two articles of varying lengths.

Let us train a new nearest neighbor model, this time with cosine distances.  We then repeat the search for Obama's 100 nearest neighbors.

In [ ]:
model2_tf_idf = turicreate.nearest_neighbors.create(wiki, label='name', features=['tf_idf'],
                                                    method='brute_force', distance='cosine')

In [ ]:
nearest_neighbors_cosine = model2_tf_idf.query(wiki[wiki['name'] == 'Barack Obama'], label='name', k=100)
nearest_neighbors_cosine = nearest_neighbors_cosine.join(wiki[['name', 'length']], on={'reference_label':'name'})

In [ ]:
nearest_neighbors_cosine.sort('rank')

From a glance at the above table, things look better.  For example, we now see Joe Biden as Barack Obama's nearest neighbor!  We also see Hillary Clinton on the list.  This list looks even more plausible as nearest neighbors of Barack Obama.

Let's make a plot to better visualize the effect of having used cosine distance in place of Euclidean on our TF-IDF vectors.

In [ ]:
plt.figure(figsize=(10.5,4.5))
plt.figure(figsize=(10.5,4.5))
plt.hist(wiki['length'], 50, color='k', edgecolor='None', histtype='stepfilled', normed=True,
         label='Entire Wikipedia', zorder=3, alpha=0.8)
plt.hist(nearest_neighbors_euclidean['length'], 50, color='r', edgecolor='None', histtype='stepfilled', normed=True,
         label='100 NNs of Obama (Euclidean)', zorder=10, alpha=0.8)
plt.hist(nearest_neighbors_cosine['length'], 50, color='b', edgecolor='None', histtype='stepfilled', normed=True,
         label='100 NNs of Obama (cosine)', zorder=11, alpha=0.8)
plt.axvline(x=wiki['length'][wiki['name'] == 'Barack Obama'][0], color='k', linestyle='--', linewidth=4,
           label='Length of Barack Obama', zorder=2)
plt.axvline(x=wiki['length'][wiki['name'] == 'Joe Biden'][0], color='g', linestyle='--', linewidth=4,
           label='Length of Joe Biden', zorder=1)
plt.axis([0, 1000, 0, 0.04])
plt.legend(loc='best', prop={'size':15})
plt.title('Distribution of document length')
plt.xlabel('# of words')
plt.ylabel('Percentage')
plt.rcParams.update({'font.size': 16})
plt.tight_layout()

Indeed, the 100 nearest neighbors using cosine distance provide a sampling across the range of document lengths, rather than just short articles like Euclidean distance provided.

**Moral of the story**: In deciding the features and distance measures, check if they produce results that make sense for your particular application.

# Problem with cosine distances: tweets vs. long articles

Happily ever after? Not so fast. Cosine distances ignore all document lengths, which may be great in certain situations but not in others. For instance, consider the following (admittedly contrived) example.

```
+--------------------------------------------------------+
|                                             +--------+ |
|  One that shall not be named                | Follow | |
|  @username                                  +--------+ |
|                                                        |
|  Democratic governments control law in response to     |
|  popular act.                                          |
|                                                        |
|  8:05 AM - 16 May 2016                                 |
|                                                        |
|  Reply   Retweet (1,332)   Like (300)                  |
|                                                        |
+--------------------------------------------------------+
```

How similar is this tweet to Barack Obama's Wikipedia article? Let's transform the tweet into TF-IDF features, using an encoder fit to the Wikipedia dataset.  (That is, let's treat this tweet as an article in our Wikipedia dataset and see what happens.)

In [ ]:
sf = turicreate.SFrame({'text': ['democratic governments control law in response to popular act']})
sf['word_count'] = turicreate.text_analytics.count_words(sf['text'])

encoder = turicreate.toolkits._feature_engineering.TFIDF(features=['word_count'], output_column_prefix='tf_idf')
encoder.fit(wiki)
sf = encoder.transform(sf)
sf

Let's look at the TF-IDF vectors for this tweet and for Barack Obama's Wikipedia entry, just to visually see their differences.

In [ ]:
tweet_tf_idf = sf[0]['tf_idf.word_count']
tweet_tf_idf

In [ ]:
obama = wiki[wiki['name'] == 'Barack Obama']
obama

Now, compute the cosine distance between the Barack Obama article and this tweet:

In [ ]:
obama_tf_idf = obama[0]['tf_idf']
turicreate.toolkits.distances.cosine(obama_tf_idf, tweet_tf_idf)

Let's compare this distance to the distance between the Barack Obama article and all of its Wikipedia 10 nearest neighbors:

In [ ]:
model2_tf_idf.query(obama, label='name', k=10)

With cosine distances, the tweet is "nearer" to Barack Obama than everyone else, except for Joe Biden!  This probably is not something we want. If someone is reading the Barack Obama Wikipedia page, would you want to recommend they read this tweet? Ignoring article lengths completely resulted in nonsensical results. In practice, it is common to enforce maximum or minimum document lengths. After all, when someone is reading a long article from _The Atlantic_, you wouldn't recommend him/her a tweet.